In [56]:
import deepxde as dde
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [57]:
# Set random seed
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
dde.backend.tf.random.set_random_seed(seed)

# Set hyperparameters
n_output = 4 # postition (x), theta 1, theta 2, force on cart (u_norm)

num_domain = 1000

n_adam = 5000

lr = 2e-2 # for Adam
loss_weights = [1. for _ in range(9)]

# Set physical parameters
tmin, tmax = 0.0, 10.0
xmin, xmax = -5.0, 5.0
target = -1. # both theta1 and theta2 should be close to 180 degrees

# Define constants
m1, m2, mc, L1, L2, LC1, LC2, I1, I2, g = [0.1, 0.1, 1, 0.5, 0.5, 0.25, 0.25, 0.01, 0.01, 9.81]  # Example values
Bc, B1, B2 = [0.5, 0.001, 0.001]  # Damping coefficients
u_max = 10  # Maximum force

In [58]:
class Custom_BC(dde.icbc.BC):
    def __init__(self, geom, func, on_boundary, component=0):
        super().__init__(geom, on_boundary, component)
        self.func = dde.icbc.boundary_conditions.npfunc_range_autocache(dde.utils.return_tensor(func))
        
    def error(self, X, inputs, outputs, beg, end, aux_var=None):
        # beg and end specify the current batch range
        values = self.func(X, beg, end, aux_var)
        theta1 = outputs[:, 1:2]
        theta2 = outputs[:, 2:3]
        goal1 = tf.cos(theta1)
        goal2 = tf.cos(theta2)

        return ((goal1[beg:end, self.component:self.component + 1] - values) ** 2) + ((goal2[beg:end, self.component:self.component + 1] - values) ** 2)

In [59]:
def ode_double_pendulum(t, u):
    # Unpack the input tensor
    x, q1, q2, u_norm = u[:, 0:1], u[:, 1:2], u[:, 2:3], u[:, 3:4]
    # u_force = u_max * tf.tanh(u_norm)  # Scale and bound the control input

    # # Compute time derivatives
    # xdot_t, q1dot_t, q2dot_t = [dde.grad.jacobian(var, t) for var in [x, q1, q2]]
    # xdot_tt, q1dot_tt, q2dot_tt = [dde.grad.jacobian(var_t, t) for var_t in [xdot_t, q1dot_t, q2dot_t]]

    # # Compute intermediates
    # h = [tf.reshape(var, (-1, 1)) for var in [mc + m1 + m2, m1 * LC1 + m2 * L1, m2 * LC2, m1 * LC1**2 + m2 * L1**2 + I1, m2 * LC2 * L1, m2 * LC2**2 + I2, m1 * LC1 * g + m2 * L1 * g, m2 * LC2 * g]]
    # B_reshaped = [tf.reshape(B, (-1, 1)) for B in [Bc, B1, B2]]

    # # Compute matrices
    # M = tf.stack([[h[0], h[1] * tf.cos(q1), h[2] * tf.cos(q2)],
    #               [h[1] * tf.cos(q1), h[3], h[4] * tf.cos(q1 - q2)],
    #               [h[2] * tf.cos(q2), h[4] * tf.cos(q1 - q2), h[5]]], axis=1)

    # zero_tensor = tf.zeros_like(B_reshaped[0])
    # C = tf.stack([[B_reshaped[0], -h[1] * q1dot_t * tf.sin(q1), -h[2] * q2dot_t * tf.sin(q2)],
    #               [zero_tensor, B_reshaped[1] + B_reshaped[2], h[4] * q2dot_t * tf.sin(q1 - q2) - B_reshaped[2]],
    #               [zero_tensor, -h[4] * q1dot_t * tf.sin(q1 - q2) - B_reshaped[2], B_reshaped[2]]], axis=1)

    # G = tf.stack([tf.zeros_like(h[6]), -h[6] * tf.sin(q1), -h[7] * tf.sin(q2)], axis=0)
    # U = tf.stack([u_force, tf.zeros_like(u_force), tf.zeros_like(u_force)], axis=1)

    # # Compute DQ, CDQ, b, and Mb
    # DQ = tf.expand_dims(tf.stack([xdot_t, q1dot_t, q2dot_t], axis=0), axis=-1)
    # CDQ = tf.matmul(C, DQ)
    # b = tf.expand_dims(tf.stack([xdot_tt, q1dot_tt, q2dot_tt], axis=0), axis=-1)
    # Mb = tf.matmul(M, b)

    # # Compute residuals
    # residual = Mb - (U - CDQ - G)
    # return residual
    return tf.zeros_like(u)

In [60]:
def initial(_, on_initial):
    return on_initial

def boundary_left(t, on_boundary):
    '''
        on_boundary is passed here by deepxde and it serves as an intial filter which tells if a point lies on the boundary or not

        np.isclose(t[0], tmin) checks if the point is on the left boundary or not and this is a second filter
    '''

    return on_boundary * np.isclose(t[0], tmin)

def boundary_right(t, on_boundary):
    '''
        on_boundary is passed here by deepxde and it serves as an intial filter which tells if a point lies on the boundary or not

        np.isclose(t[0], tmax) checks if the point is on the right boundary or not and this is a second filter
    '''

    return on_boundary * np.isclose(t[0], tmax)

In [61]:
geom = dde.geometry.TimeDomain(tmin, tmax)

# INITIAL CONDITIONS
position_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=0) # posittion = 0 at time = 0
theta2_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=1) # theta1 = 0 at time = 0
theta1_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=2) # theta2 = 0 at time = 0
force_initial = dde.icbc.IC(geom, lambda t: np.array([0.]), initial, component=3) # force = 0 at time = 0

# NEUMANN CONDITIONS
velocity_initial = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=0) # cart velocity 1 = 0 at time = 0
angular_velocity1_initial = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=1) # angular velocity 1 = 0 at time = 0
angular_velocity2_initial = dde.icbc.NeumannBC(geom, lambda t: np.array([0.]), boundary_left, component=2) # angular velocity 2 = 0 at time = 0

# CUSTOM BOUNDARY CONDITIONS - GOAL AND POSITION RANGE
goal = Custom_BC(geom, lambda t: np.array([target]), boundary_right) # custom ICBC

losses = [position_initial, theta1_initial, theta2_initial, force_initial, velocity_initial, angular_velocity1_initial, angular_velocity2_initial, goal]

data = dde.data.PDE(geom, ode_double_pendulum, losses, num_domain=num_domain, num_boundary=2)
# dataset size here will be 1002 (1000 domain + 2 boundary)

In [62]:
net = dde.nn.FNN([1] + [64] * 3 + [n_output], "tanh", "Glorot normal")

In [63]:
resampler = dde.callbacks.PDEPointResampler(period=100)

In [64]:
model = dde.Model(data, net)
model.compile("adam", lr=lr, loss_weights=loss_weights)

Compiling model...
Building feed-forward neural network...
'build' took 0.121496 s



/Users/arjundosajh/miniconda3/envs/ml/lib/python3.11/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 0.643678 s



In [65]:
losshistory, train_state = model.train(display_every=100, iterations=n_adam, callbacks=[resampler])

Training model...

Step      Train loss                                                                                    Test loss                                                                                     Test metric
0         [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.23e-03, 5.95e-03, 1.72e-02, 5.75e+01]    [0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 1.23e-03, 5.95e-03, 1.72e-02, 5.75e+01]    []  
100       [0.00e+00, 3.42e-05, 2.42e-05, 1.58e-05, 8.76e-07, 2.41e-05, 9.53e-06, 2.34e-05, 3.81e-07]    [0.00e+00, 3.42e-05, 2.42e-05, 1.58e-05, 8.76e-07, 2.41e-05, 9.53e-06, 2.34e-05, 3.81e-07]    []  
200       [0.00e+00, 4.49e-10, 3.29e-09, 3.55e-11, 1.37e-09, 2.42e-11, 4.57e-10, 1.80e-10, 4.22e-07]    [0.00e+00, 4.49e-10, 3.29e-09, 3.55e-11, 1.37e-09, 2.42e-11, 4.57e-10, 1.80e-10, 4.22e-07]    []  
300       [0.00e+00, 9.03e-07, 1.86e-02, 1.79e-03, 6.54e-03, 1.43e-05, 6.31e-07, 1.67e-07, 6.22e-07]    [0.00e+00, 9.03e-07, 1.86e-02, 1.79e-03, 6.54e-03, 1.43e-0

KeyboardInterrupt: 